---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [40]:
import pandas as pd
def get_list_of_university_towns():
    '''
    Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )
    '''
    with open('university_towns.txt') as file:
        data = [line[:-1] for line in file]
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1].replace('–', '-')
            state_town.append([state,town])
        else:
            town = line
            state_town.append([state,town])
    
    column_names = ['State','RegionName']
    university_towns = pd.DataFrame(state_town,columns=column_names)
    return university_towns
        

get_list_of_university_towns().shape

(517, 2)

In [42]:

def get_gdp():
    import pandas as pd
    column_names = ['Quarter', 'GDP']
    column_used = [4, 5]
    gdp = pd.read_excel('gdplev.xls', skiprows=7, parse_cols=column_used, names=column_names)
    index_for_2000q1 = gdp[gdp['Quarter'] == '2000q1'].index[0]
    gdp = gdp.loc[index_for_2000q1:]
    return gdp

In [43]:
def get_recession_start():
    '''
    Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    '''
    import pandas as pd
    gdp = get_gdp()
    for index, value in enumerate(gdp.values):
        if (value[1] > gdp.iloc[index+1][1]) & (gdp.iloc[index+1][1] > gdp.iloc[index+2][1]):
            return gdp.iloc[index+1][0]
    else:
        return None

get_recession_start()

u'2008q4'

In [44]:
def get_recession_end():
    '''
    Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3
    '''
    gdp = get_gdp()
    ression_start_quarter = get_recession_start()
    ression_start_quarter_index = gdp[gdp['Quarter'] == ression_start_quarter].index[0]
    gdp_at_ression_start = gdp.loc[ression_start_quarter_index:]
    for index, value in enumerate(gdp_at_ression_start.values):
        if (value[1] < gdp_at_ression_start.iloc[index+1][1]) & (gdp_at_ression_start.iloc[index+1][1] < gdp_at_ression_start.iloc[index+2][1]):
            return gdp_at_ression_start.iloc[index+1][0]
    else:
        return None

get_recession_end()

u'2009q3'

In [45]:
def get_recession_bottom():
    '''
    Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3
    '''
    gdp = get_gdp()
    ression_start_quarter = get_recession_start()
    ression_end_quarter = get_recession_end()
    ression_start_quarter_index = gdp[gdp['Quarter'] == ression_start_quarter].index[0]
    ression_end_quarter_index = gdp[gdp['Quarter'] == ression_end_quarter].index[0]
    ression_period = gdp.loc[ression_start_quarter_index: ression_end_quarter_index]
    ression_bottom = ression_period.loc[ression_period['GDP'].idxmin()]['Quarter']
    
    return ression_bottom

get_recession_bottom()

u'2009q2'

In [46]:
def convert_housing_data_to_quarters():
    '''
    Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    years = xrange(2000, 2017)
    quarters = ['q1', 'q2', 'q3', 'q4']
    year_quarters = []
    for year in years:
        for quarter in quarters:
            year_quarters.append('{}{}'.format(year, quarter))
#     print(year_quarters)
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    print house_df.columns
    
    return house_df

convert_housing_data_to_quarters().head()

Index([u'RegionID', u'RegionName', u'State', u'Metro', u'CountyName',
       u'SizeRank', u'1996-04', u'1996-05', u'1996-06', u'1996-07',
       ...
       u'2015-11', u'2015-12', u'2016-01', u'2016-02', u'2016-03', u'2016-04',
       u'2016-05', u'2016-06', u'2016-07', u'2016-08'],
      dtype='object', length=251)


,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (True, 0.005496427353694603, 'university town')